LINEAR SYSTEMS

In [ ]:
import numpy as np

In [ ]:
from numpy import linalg

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

In [ ]:
import sympy as sp

In [ ]:
def sassenfeld(A):
    """
    Function to check the convergence condition of a system using the Sassenfeld criterion.
    The function returns True if the system converges and False otherwise.
    """
    n = np.shape(A)[0]  # Get the number of rows (or columns) of the matrix A
    B = np.zeros(n)  # Initialize the B vector with zeros

    # Initialize the first value of B
    B[0] = np.sum(np.abs(A[0])) - np.abs(A[0][0])

    # Calculate the values of the B vector for each row
    for i in range(1, n):
        a = 0
        for j in range(0, n):
            # Check if i != j and a < j
            if i != j and a < j:
                B[i] += abs(A[i][j]) * B[a]  # Accumulate the product of the coefficients
            elif i != j:
                B[i] += abs(A[i][j])  # Accumulate the absolute values of the coefficients

    # Get the maximum value of B
    max_x = max(B)
    
    return max_x < 1  # Return True if the sum is less than 1 (convergence), otherwise return False


In [ ]:
def converge(A):
    # Get the number of rows (or columns) of the matrix A
    n = np.shape(A)[0]

    # Iterate through each row of the matrix
    for i in range(n):
        row_sum = 0  # Initialize the sum of the row (excluding the diagonal element)
        
        # Iterate through each column of the matrix
        for j in range(n):
            if i != j:  # Exclude the diagonal element
                row_sum += A[i][j]  # Add the off-diagonal elements to the row sum
        
        # Check if the absolute value of the diagonal element is less than the row sum
        if abs(A[i][i]) < row_sum:
            print("Row sum:", row_sum)
            print("Diagonal element:", A[i][i])
            return False  # Return False if the condition is not satisfied

    # Return True if the matrix satisfies the convergence condition
    return True

In [ ]:
def jacobi(A, b, x0, tol, N):
    # Check if the matrix satisfies the convergence condition
    c = converge(A)
    if not c:
        # If not, check using the Sanssenfeld criterion
        c = sanssenfeld(A)
        if not c:
            return "The system does not converge."

    # Get the number of rows (or columns) of the matrix A
    n = np.shape(A)[0]
    # Initialize the solution vector with zeros
    x = np.zeros(n)
    # Initialize the iteration counter
    it = 0

    # Iterative process
    while it < N:
        it += 1  # Increment the iteration counter

        # Jacobi iteration
        for i in range(n):
            x[i] = b[i]  # Start with the corresponding value from vector b
            for j in range(n):
                if i != j:  # Exclude the diagonal element
                    x[i] -= A[i, j] * x0[j]  # Subtract the product of A[i, j] and the previous solution
            x[i] /= A[i, i]  # Divide by the diagonal element

        # Check if the tolerance condition is satisfied
        if np.linalg.norm(x - x0, np.inf) < tol:
            return x  # Return the solution if the tolerance is met

        # Prepare for the next iteration
        x0 = np.copy(x)

    # Return the solution after the maximum number of iterations
    return x

In [ ]:
def gauss_seidel(A, b, x0, tol, N):
    # Get the number of rows (or columns) of the matrix A
    n = len(A)
    # Initialize the solution vector with the initial guess
    x = np.copy(x0)
    # List to store the points of each iteration
    iteration_points = []
    
    # Iterate up to the maximum number of iterations
    for it in range(N):
        # Store the previous solution for comparison
        x_old = np.copy(x)
        
        # Perform the Gauss-Seidel iteration
        for i in range(n):
            # Start with the corresponding value from vector b
            sum_ = b[i]
            for j in range(n):
                if i != j:  # Exclude the diagonal element
                    sum_ -= A[i, j] * x[j]  # Subtract the product of A[i, j] and the current solution
            x[i] = sum_ / A[i, i]  # Divide by the diagonal element
        
        # Store the current solution in the iteration points list
        iteration_points.append(np.copy(x))
        
        # Check if the tolerance condition is satisfied
        if np.linalg.norm(x - x_old, np.inf) < tol:
            return x, iteration_points  # Return the solution and iteration points if the tolerance is met
    
    # Return the solution and iteration points after the maximum number of iterations
    return x, iteration_points

In [ ]:
def calculate_errors(A, b, x0, tol, N):
    # Compute the exact solution of the system
    x_exact = np.linalg.solve(A, b)
    
    # Solve the system using the Jacobi method and get the iteration points
    x_jacobi, jacobi_points = jacobi(A, b, x0, tol, N)
    
    # Solve the system using the Gauss-Seidel method and get the iteration points
    x_gauss, gauss_points = gauss_seidel(A, b, x0, tol, N)
    
    # Initialize a table to store the iteration number and errors
    table = []
    
    # Iterate through the points of both methods (up to the minimum number of iterations)
    for i in range(min(len(jacobi_points), len(gauss_points))):
        # Compute the error for the Jacobi method at the current iteration
        jacobi_error = np.linalg.norm(jacobi_points[i] - x_exact, np.inf)
        
        # Compute the error for the Gauss-Seidel method at the current iteration
        gauss_error = np.linalg.norm(gauss_points[i] - x_exact, np.inf)
        
        # Append the iteration number and errors to the table
        table.append([i + 1, jacobi_error, gauss_error])
    
    # Create a DataFrame to display the errors in a tabular format
    df = pd.DataFrame(table, columns=["Iteration", "Jacobi Error", "Gauss-Seidel Error"])
    return df

In [ ]:
def main():
    # Define the system of equations: Ax = b
    A = np.array([[80, 0, 30, 10], [0, 80, 10, 10], [16, 20, 60, 72], [4, 0, 0, 8]], dtype=float)  # Coefficient matrix
    b = np.array([40, 27, 31, 2], dtype=float)  # Independent terms vector
    x0 = np.array([0.5, 0.2, 0.2, 0], dtype=float)  # Initial guess for the solutions
    tol = 0.000001  # Tolerance for the convergence of the Jacobi method
    N = 100

    solution = jacobi(A, b, x0, tol, N)
    print("Solution found:", solution)

# Run the program
if __name__ == "__main__":
    main()